In [1]:
import pandas as pd
esun = pd.read_csv('投顧推薦.csv')
esun.drop(['Team','member','Products','stock_name','recomment_date','closing_price','current_price','RofR','udt'],axis = 1,inplace = True)

In [2]:
esun['Datadt'] = esun['Datadt'].astype(str)

In [3]:
esun['date_wk'] = pd.to_datetime(esun['Datadt'])

In [4]:
esun['date_wk'] = esun['date_wk'] + pd.tseries.offsets.Week(weekday=0)

In [5]:
esun['date_wk'] = esun['date_wk'] - pd.tseries.offsets.Week(n=2)

In [6]:
stock_status = pd.read_pickle('stock_status_new.pk')

In [7]:
stock_status.columns

Index(['stock', 'date', 'up_or_down', 'up_or_down_150'], dtype='object')

In [8]:
stock_status['stock_id'] = stock_status['stock']
stock_status.drop(['stock','up_or_down'],axis=1,inplace=True)

In [9]:
stock_status['date'] = pd.to_datetime(stock_status['date'])

In [10]:
stock_status['date_wk'] = stock_status['date'] - pd.tseries.offsets.Week(weekday=0)

In [11]:
esun_final = pd.merge(esun,stock_status,on=['stock_id','date_wk'],how='left')
#esun.iloc[1200:1300,:]

In [12]:
esun_final = esun_final.dropna()

In [13]:
esun_final.columns[4]

'up_or_down_150'

In [14]:
esun_final.rename(columns={ esun_final.columns[4]: "y_true" }, inplace=True)

In [15]:
esun_final = esun_final.drop_duplicates()
esun_final.shape

(1661, 5)

In [16]:
list_150 = pd.read_excel("150_list.xlsx")

In [17]:
list_150.rename(columns={ list_150.columns[0]: "stock_id" }, inplace=True)

In [18]:
list_150['stock_id'] = list_150['stock_id'].astype(str)

In [19]:
esun_final_150 = pd.merge(esun_final,list_150,on=['stock_id'],how='inner')

In [20]:
esun_final_150.head()

,Datadt,stock_id,date_wk,date,y_true
0,20160104,2106,2015-12-28,2015-12-31,1.0
1,20160111,2106,2016-01-04,2016-01-08,1.0
2,20160118,2106,2016-01-11,2016-01-15,0.0
3,20160125,2106,2016-01-18,2016-01-22,0.0
4,20160201,2106,2016-01-25,2016-01-30,0.0


In [21]:
# esun_final['y_pred'] = 1
# esun_final['y_true'] = esun_final['y_true'].astype(int)
esun_final_150['y_pred'] = 1
esun_final_150['y_true'] = esun_final_150['y_true'].astype(int)

In [22]:
# esun_final_test = esun_final[esun_final['date_wk'].between("2018-07-01","2018-09-30")]
esun_final_test_150 = esun_final_150[esun_final_150['date_wk'].between("2018-07-01","2018-09-30")]

In [23]:
# esun_final_test.date_wk.value_counts()
esun_final_test_150.date_wk.value_counts()

2018-08-27    12
2018-09-17    11
2018-09-10    11
2018-08-20    11
2018-08-13    10
2018-08-06    10
2018-07-30     9
2018-07-23     9
2018-07-16     9
2018-07-09     8
2018-07-02     8
Name: date_wk, dtype: int64

In [29]:
from sklearn import metrics
from sklearn.metrics import classification_report
precision = metrics.precision_score(esun_final_150['y_true'], esun_final_150['y_pred'], average='micro')
print(precision)
recall = metrics.recall_score(esun_final_150['y_true'], esun_final_150['y_pred'], average='micro')
print(recall)
f1 = metrics.f1_score(esun_final_150['y_true'], esun_final_150['y_pred'], average='micro')
print(f1)
print(2*((precision*recall)/(precision+recall)))
print(classification_report(esun_final_150['y_true'],esun_final_150['y_pred'],digits=4))

0.5171583383503914
0.5171583383503914
0.5171583383503914
0.5171583383503914
             precision    recall  f1-score   support

          0     0.0000    0.0000    0.0000       802
          1     0.5172    1.0000    0.6817       859

avg / total     0.2675    0.5172    0.3526      1661



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [30]:
precision = metrics.precision_score(esun_final_test_150['y_true'], esun_final_test_150['y_pred'], average='micro')
print(precision)
recall = metrics.recall_score(esun_final_test_150['y_true'], esun_final_test_150['y_pred'], average='micro')
print(recall)
f1 = metrics.f1_score(esun_final_test_150['y_true'], esun_final_test_150['y_pred'], average='micro')
print(f1)
print(2*((precision*recall)/(precision+recall)))
print(classification_report(esun_final_test_150['y_true'],esun_final_test_150['y_pred'],digits=4))

0.6018518518518519
0.6018518518518519
0.6018518518518519
0.6018518518518519
             precision    recall  f1-score   support

          0     0.0000    0.0000    0.0000        43
          1     0.6019    1.0000    0.7514        65

avg / total     0.3622    0.6019    0.4523       108



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
